<a href="https://colab.research.google.com/github/Bentley97/NLU_First_assignment/blob/main/FirstAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import spacy

test_sentece = "I saw a man with a telescope"

nlp = spacy.load('en_core_web_sm')
doc = nlp(test_sentece)

print(test_sentece)

for sent in spacy_doc.sents:
    for token in sent:
        print("{}\t{}\t{}\t{}".format(token.i, token.text, token.head, token.dep_))

I saw a man with a telescope
0	I	saw	nsubj
1	saw	saw	ROOT
2	a	man	det
3	man	saw	dobj
4	with	man	prep
5	a	telescope	det
6	telescope	with	pobj
